# Calculating the Mulliken population

In [Cedillo2012](https://link.springer.com/article/10.1007/s00214-012-1227-6), we find a Mulliken charge of 0.20 on the carbon atom in CO. In this notebook, we will reproduce this result.

In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

Let's start by creating the molecule, and quantizing the Hamiltonian in the associated spinor basis.

In [2]:
molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/CO_mulliken.xyz" , 0)  # create a neutral molecule
N = molecule.numberOfElectrons()

spinor_basis = gqcpy.RSpinorBasis(molecule, "STO-3G")
S = spinor_basis.quantizeOverlapOperator().parameters()

sq_hamiltonian = gqcpy.SQHamiltonian.Molecular(spinor_basis, molecule)

Afterwards, we should solve the RHF SCF equations.

In [3]:
environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, sq_hamiltonian, S)
solver = gqcpy.RHFSCFSolver.DIIS()
objective = gqcpy.DiagonalRHFFockMatrixObjective(sq_hamiltonian)  # use the default threshold of 1.0e-08

rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()
D = rhf_parameters.calculateScalarBasis1RDM()  # the AO density matrix

The Mulliken population is effectively the expectation value of the Mulliken operator and a density matrix. Calculating the Mulliken operator in the AO basis is task for the `SpinorBasis`. All it needs is the indices of the spinors that are associated with the carbon atom.

In [4]:
C_indices = [0, 1, 2, 3, 4]

mulliken_operator = spinor_basis.calculateMullikenOperator(C_indices)
C_population = mulliken_operator.calculateExpectationValue(D)[0]

Once we have the Mulliken population, the Mulliken charge can be calculated straightforwardly.

In [5]:
C_charge = 6 - C_population
print(C_charge)

0.19893362956575533
